In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_auc_score, recall_score, classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, recall_score
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

In [38]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

In [39]:
# import libraries for visualizations

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [40]:
df = pd.read_excel (r'/Users/MaeganBoosten/Thesis_Project/Final/Original_SHAP.xlsx')
print (df)

     duration  credit_amount  intallment_rate  residence_since       age  \
0    0.256953       1.078480         0.918477        -0.765977 -0.751642   
1   -0.240857      -0.237923         0.918477         1.046987  2.590552   
2    0.505858      -0.248556         0.024147         0.140505 -1.279356   
3    0.256953       1.168508        -0.870183        -0.765977 -1.279356   
4    2.248194       1.165673         0.024147        -0.765977 -1.015499   
..        ...            ...              ...              ...       ...   
369  1.252574       1.757239        -0.870183        -0.765977 -0.839594   
370 -0.738668      -0.877693         0.918477         1.046987  1.535122   
371 -0.738668      -0.912429         0.918477        -0.765977  0.919455   
372  0.256953      -0.728473         0.918477         1.046987  0.215835   
373 -0.738668      -0.615760        -1.764514        -1.672459 -1.103451   

     number_of_existcr  number_of_dependents  telephon   foreign  \
0             1.027

In [41]:
#split dataset

In [42]:
x, y = df.drop('False Positive', axis=1), df['False Positive']
x.shape, y.shape

((374, 114), (374,))

In [43]:
#Oversampling
# summarize class distribution
print(Counter(y))
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(x, y)
# summarize class distribution
print(Counter(y_over))

Counter({0: 270, 1: 104})
Counter({1: 270, 0: 270})


In [44]:
x_train, x_test, y_train, y_test= train_test_split(X_over,y_over, test_size=.5, random_state=42)

In [45]:
x_train.shape, x_test.shape

((270, 114), (270, 114))

In [46]:
#Number of False Positives in y_train
sum(y_train)

136

In [47]:
#Number of False Positives in y_test
sum(y_test)

134

In [48]:
#Baseline for evaluation
y.value_counts(normalize=True)

0    0.721925
1    0.278075
Name: False Positive, dtype: float64

In [49]:
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)

RandomForestClassifier()

In [50]:
y_pred=clf.predict(x_test)

In [51]:
def acc_val(TP, TN, FP, FN, data):    
    accuracy = (TP + TN) / float(TP + TN + FP + FN)
    recall = TP / float(FN + TP)
    specificity = TN / (TN + FP)
    false_positive_rate = FP / float(TN + FP)
    precision = TP / float(TP + FP)
    F1 = 2 * (precision * recall) / (precision + recall)
    print('The values for {} are:'.format(data))
    print('Accuracy: {} ,Recall: {} , Specificity: {} , False Positive Rate: {} , Precision: {}, F1 Score: {}'.format(accuracy, recall, specificity, false_positive_rate,
                                                                                                                   precision, F1))

In [52]:
confusion_matrix(y_test, clf.predict(x_test))

array([[117,  19],
       [ 32, 102]])

In [53]:
print(classification_report(y_test, clf.predict(x_test)))

              precision    recall  f1-score   support

           0       0.79      0.86      0.82       136
           1       0.84      0.76      0.80       134

    accuracy                           0.81       270
   macro avg       0.81      0.81      0.81       270
weighted avg       0.81      0.81      0.81       270



In [54]:
confusion = metrics.confusion_matrix(y_test, clf.predict(x_test))
print(confusion)
TP1 = confusion[1, 1]
TN1 = confusion[0, 0]
FP1 = confusion[0, 1]
FN1 = confusion[1, 0]
acc_val(TP1, TN1, FP1, FN1, 'Test Data')

[[117  19]
 [ 32 102]]
The values for Test Data are:
Accuracy: 0.8111111111111111 ,Recall: 0.7611940298507462 , Specificity: 0.8602941176470589 , False Positive Rate: 0.13970588235294118 , Precision: 0.8429752066115702, F1 Score: 0.7999999999999999


In [55]:
roc_auc_score(y_test, clf.predict(x_test))

0.8107440737489027

In [56]:
# Hyperparameter tuning

In [57]:
# create model with default parameters- baseline
skf= StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
rf_baseline = RandomForestClassifier(random_state=42, n_jobs=-1)

# Train it on the training set
cv_result_baseline= cross_val_score(rf_baseline, x_test, y_test, cv=skf)

# Evalute the results (cross-val)
print("CV accuracy score: {:.2f}%".format(cv_result_baseline.mean() * 100))

CV accuracy score: 78.15%


In [58]:
# train model 
rf_baseline.fit(x_train, y_train)

pred_test_rf= rf_baseline.predict(x_test)
print("Test accuracy score: {:.2f}%".format((accuracy_score(pred_test_rf, y_test) * 100)))


Test accuracy score: 77.04%


In [59]:
#Improve results, start with number of trees

In [60]:
# Create lists to save the values of accuracy on training and test sets
train_acc = []
test_acc = []
temp_train_acc = []
trees_grid = [5, 10, 15, 20, 30, 50, 75, 100]

In [61]:
for ntrees in trees_grid:
    rf = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1).fit(x_train, y_train)
    temp_train_acc=cross_val_score(rf, x_test, y_test, cv=skf)
    train_acc.append(temp_train_acc.mean())
    test_acc.append(accuracy_score(rf.predict(x_test), y_test))

In [62]:
print("Best CV accuracy is {:.2f}% with {} trees".format(max(test_acc)*100, 
                                                        trees_grid[np.argmax(test_acc)]))
plt

Best CV accuracy is 78.52% with 30 trees


<module 'matplotlib.pyplot' from '/Users/MaeganBoosten/opt/anaconda3/lib/python3.8/site-packages/matplotlib/pyplot.py'>

In [63]:
# Create lists to save the values of accuracy on training and test sets
train_acc = []
test_acc = []
temp_train_acc
max_depth_grid = [3, 5, 7, 9, 11, 13, 15, 17, 20, 22, 24]

for max_depth in max_depth_grid:
    rf = RandomForestClassifier(n_estimators=20, random_state=42, n_jobs=-1, max_depth=max_depth).fit(x_train, y_train)
    temp_train_acc=cross_val_score(rf, x_test, y_test, cv=skf)
    train_acc.append(temp_train_acc.mean())
    test_acc.append(accuracy_score(rf.predict(x_test), y_test))

print("Best CV accuracy is {:.2f}% with {} max_depth".format(max(test_acc)*100, 
                                                        max_depth_grid[np.argmax(test_acc)]))

Best CV accuracy is 77.78% with 7 max_depth


In [64]:
# Create lists to save the values of accuracy on training and test sets
train_acc = []
test_acc = []
temp_train_acc
min_samples_leaf_grid = [1, 3, 5, 7, 9, 11, 13, 15, 17, 20, 22, 24]

for min_sample in min_samples_leaf_grid:
    rf = RandomForestClassifier(n_estimators=20, random_state=42, n_jobs=-1, min_samples_leaf=min_sample).fit(x_train, y_train)
    temp_train_acc=cross_val_score(rf, x_test, y_test, cv=skf)
    train_acc.append(temp_train_acc.mean())
    test_acc.append(accuracy_score(rf.predict(x_test), y_test))

print("Best CV accuracy is {:.2f}% with {} min_sample_leaf".format(max(test_acc)*100, 
                                                        min_samples_leaf_grid[np.argmax(test_acc)]))


Best CV accuracy is 76.67% with 1 min_sample_leaf


In [65]:
# Create lists to save the values of accuracy on training and test sets
train_acc = []
test_acc = []
temp_train_acc
max_features_grid = [2, 4, 6, 8, 10, 12, 14, 16]

for max_feature in max_features_grid:
    rf = RandomForestClassifier(n_estimators=20, random_state=42, n_jobs=-1, max_features=max_feature).fit(x_train, y_train)
    temp_train_acc=cross_val_score(rf, x_test, y_test, cv=skf)
    train_acc.append(temp_train_acc.mean())
    test_acc.append(accuracy_score(rf.predict(x_test), y_test))

print("Best CV accuracy is {:.2f}% with {} max_features".format(max(test_acc)*100, 
                                                        max_features_grid[np.argmax(test_acc)]))

Best CV accuracy is 81.11% with 16 max_features


In [66]:
# Initialize the set of parameters for exhaustive search and fit 
parameters = {'max_features': [4], 
              'min_samples_leaf': [1], 
              'max_depth': [9]}
rf = RandomForestClassifier(n_estimators=20, random_state=42, n_jobs=-1)
gcv = GridSearchCV(rf, parameters, n_jobs=-1, cv=skf, verbose=1)
gcv.fit(x_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(n_estimators=20, n_jobs=-1,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [9], 'max_features': [4],
                         'min_samples_leaf': [1]},
             verbose=1)

In [67]:
gcv.best_params_, gcv.best_score_

({'max_depth': 9, 'max_features': 4, 'min_samples_leaf': 1},
 0.8111111111111111)

In [68]:
# Initialize the set of parameters for exhaustive search and fit 
parameters = {'max_features': [4], 
              'min_samples_leaf': [1], 
              'max_depth': [9]}
rf = RandomForestClassifier(n_estimators=20, random_state=42, n_jobs=-1)
gcv = GridSearchCV(rf, parameters, n_jobs=-1, cv=skf, verbose=1)
gcv.fit(x_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(n_estimators=20, n_jobs=-1,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [9], 'max_features': [4],
                         'min_samples_leaf': [1]},
             verbose=1)

In [69]:
roc_auc_score(y_test, gcv.predict(x_test))

0.7667910447761195

In [70]:
recall_score(y_test, gcv.predict(x_test))

0.7835820895522388

In [71]:
print(classification_report(y_test, gcv.predict(x_test)))

              precision    recall  f1-score   support

           0       0.78      0.75      0.76       136
           1       0.76      0.78      0.77       134

    accuracy                           0.77       270
   macro avg       0.77      0.77      0.77       270
weighted avg       0.77      0.77      0.77       270



In [72]:
confusion = metrics.confusion_matrix(y_test, gcv.predict(x_test))
print(confusion)
TP1 = confusion[1, 1]
TN1 = confusion[0, 0]
FP1 = confusion[0, 1]
FN1 = confusion[1, 0]
acc_val(TP1, TN1, FP1, FN1, 'Test Data')

[[102  34]
 [ 29 105]]
The values for Test Data are:
Accuracy: 0.7666666666666667 ,Recall: 0.7835820895522388 , Specificity: 0.75 , False Positive Rate: 0.25 , Precision: 0.7553956834532374, F1 Score: 0.7692307692307693
